In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau

In [2]:
classifier = Sequential()

classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = "same", activation = 'relu', input_shape = (28, 28,1)))
classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.5))
               
classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.5))
               
classifier.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = "same", activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.5))


classifier.add(Flatten())
               
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 50, activation = 'softmax'))

opt=tf.keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)              
classifier.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [3]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 128)       147584    
                                                        

In [4]:
for layer in classifier.layers:
    print(layer.output_shape)

(None, 28, 28, 64)
(None, 28, 28, 64)
(None, 14, 14, 64)
(None, 14, 14, 64)
(None, 14, 14, 128)
(None, 14, 14, 128)
(None, 7, 7, 128)
(None, 7, 7, 128)
(None, 7, 7, 256)
(None, 7, 7, 256)
(None, 3, 3, 256)
(None, 3, 3, 256)
(None, 2304)
(None, 128)
(None, 50)


In [10]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.Collecting split-folders



You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [11]:
import splitfolders

In [12]:
splitfolders.ratio("DataSet/Train", output="DataSet/main", seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

In [5]:
train_folder = "Dataset/main/train"
val_folder = "Dataset/main/val"
test_folder = "Dataset/test"

In [6]:
Image.LOAD_TRUNCATED_IMAGES = True


train_datagen = ImageDataGenerator(shear_range = .2, rotation_range = 25)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_set = train_datagen.flow_from_directory(train_folder, target_size = (28, 28), 
                                                 batch_size = 32, class_mode = 'categorical',color_mode="grayscale")
val_set = val_datagen.flow_from_directory(val_folder, target_size = (28, 28), 
                                                 batch_size = 32, class_mode = 'categorical',color_mode="grayscale")
test_set = test_datagen.flow_from_directory(test_folder, target_size = (28, 28), 
                                                 batch_size = 32, class_mode = 'categorical',color_mode="grayscale")

Found 10800 images belonging to 50 classes.
Found 1200 images belonging to 50 classes.
Found 3000 images belonging to 50 classes.


In [10]:
len(train_set)

338

In [12]:
len(val_set)

38

In [7]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

In [9]:
classifier.fit(train_set, steps_per_epoch = len(train_set), epochs = 10,
                        validation_data = val_set, validation_steps = len(val_set), callbacks=[reduce_lr])

Epoch 1/10
338/338 [==============================] - 236s 699ms/step - loss: 0.3919 - accuracy: 0.8799 - val_loss: 0.2035 - val_accuracy: 0.9492 - lr: 1.0000
Epoch 2/10
338/338 [==============================] - 247s 730ms/step - loss: 0.3784 - accuracy: 0.8831 - val_loss: 0.1914 - val_accuracy: 0.9450 - lr: 1.0000
Epoch 3/10
338/338 [==============================] - 211s 624ms/step - loss: 0.3294 - accuracy: 0.8963 - val_loss: 0.2008 - val_accuracy: 0.9483 - lr: 1.0000
Epoch 4/10
338/338 [==============================] - 211s 625ms/step - loss: 0.3175 - accuracy: 0.9016 - val_loss: 0.1714 - val_accuracy: 0.9483 - lr: 1.0000
Epoch 5/10
338/338 [==============================] - 213s 629ms/step - loss: 0.2952 - accuracy: 0.9054 - val_loss: 0.1834 - val_accuracy: 0.9517 - lr: 1.0000
Epoch 6/10
338/338 [==============================] - 204s 604ms/step - loss: 0.2732 - accuracy: 0.9135 - val_loss: 0.1857 - val_accuracy: 0.9525 - lr: 1.0000
Epoch 7/10
338/338 [==========================

In [9]:
classifier.evaluate(test_set)

3000/3000 [==============================] - 29s 9ms/step - loss: 1.8621 - accuracy: 0.4877


[1.8620712757110596, 0.48766666650772095]